In [1]:
!pip install pulp -q

## 최적화 문제 vs. 실행 가능성 문제
- 최적화 문제(Optimization Problem): 목적식을 최소화하거나 최대화하는 해를 찾습니다. 예를 들어, "가장 짧은 경로를 찾아라", "가장 적은 재료를 써서 제품을 만들어라" 같은 문제가 최적화 문제입니다. 이때 목적식은 의미 있는 값을 가지며, 우리가 원하는 최종 목표가 됩니다.

- 실행 가능성 문제(Feasibility Problem): 목적식은 중요하지 않고, 모든 제약 조건을 만족하는 해가 존재하는지 여부만 확인합니다. 즉, "조건에 맞는 해가 하나라도 있는가?"가 핵심입니다. : 즉 Z = 0
- 목적식 수정 가능
> - 강의실 활용률 최대화: 모든 시간 슬롯과 강의실의 조합 중 사용된 수를 최대화합니다.
> - 교수 선호 시간대 배정: 교수들이 선호하는 시간대에 강의를 배정했을 때 얻는 점수를 최대화합니다.
> - 빈 강의실 최소화: 사용되지 않는 강의실 시간대를 최소화합니다.

## 강의 시간표 배정 문제의 정수계획법 모델

제공된 코드는 강의, 시간대, 강의실에 대한 제약 조건을 만족하는 실행 가능한 시간표를 찾는 **실행 가능성 문제(Feasibility Problem)**를 모델링합니다.

---
## 1. 집합 (Sets)

- $C$: 모든 교과목의 집합 ($i \in C$)  
- $T$: 모든 시간대의 집합 ($t \in T$)  
- $R$: 모든 강의실의 집합 ($r \in R$)  
- $P$: 모든 교수의 집합 ($p \in P$)  
- $C_p$: 특정 교수 $p$가 담당하는 교과목들의 집합 ($C_p \subseteq C$)  


---
## 2. 변수 및 매개변수 (Variables and Parameters)

### 결정 변수 (Decision Variable)

교과목 $i$가 시간대 $t$에 강의실 $r$에 배정되었는지를 나타내는 변수:

$$
x_{i,t,r} =
\begin{cases}
1 & \text{교과목 $i$가 시간대 $t$에 강의실 $r$에 배정된 경우} \\
0 & \text{그 외의 경우}
\end{cases}
$$

$$
x_{i,t,r} \in \{0,1\}, \quad \forall i \in C, \forall t \in T, \forall r \in R
$$

---

### 매개변수 (Parameters)

- $h_i$: 교과목 $i$의 필수 시수


---

## 3. 목적식 (Objective Function)

본 모델은 최적화가 아닌 실행 가능성 문제이므로, 단순히 제약조건을 만족하는 해를 찾는 것이 목표입니다.  
따라서 목적식은 다음과 같이 표현됩니다.

$$
\text{Minimize } Z = 0
$$

---

## 4. 제약 조건 (Constraints)

### 제약 조건 1: 시수 제약
각 교과목은 정해진 시수만큼 정확히 배정되어야 한다.  

$$
\sum_{t \in T} \sum_{r \in R} x_{i,t,r} = h_i, \quad \forall i \in C
$$

---

### 제약 조건 2: 강의실 동시 사용 제약
특정 시간대에 한 강의실은 오직 하나의 교과목에만 사용될 수 있다.  

$$
\sum_{i \in C} x_{i,t,r} \leq 1, \quad \forall t \in T, \forall r \in R
$$

---

### 제약 조건 3: 교수 동시 강의 제약
각 교수는 특정 시간대에 하나의 교과목만 강의할 수 있다.  

$$
\sum_{i \in C_p} \sum_{r \in R} x_{i,t,r} \leq 1, \quad \forall t \in T, \forall p \in P
$$

---

### 제약 조건 4: 교과목 동시 배정 제약
각 교과목은 특정 시간대에 하나의 강의실에만 배정될 수 있다.  

$$
\sum_{r \in R} x_{i,t,r} \leq 1, \quad \forall i \in C, \forall t \in T
$$


In [2]:
import pulp
import numpy as np
from itertools import product

def create_courses():
    """테스트용 교과목 데이터"""
    courses = [
        {"name": "알고리즘", "hours": 3, "professor": "김민준", "type": "컴퓨터", "grade": 3, "students": 25},
        {"name": "데이터베이스", "hours": 2, "professor": "이서연", "type": "일반", "grade": 2, "students": 35},
        {"name": "운영체제", "hours": 3, "professor": "최유진", "type": "컴퓨터", "grade": 3, "students": 28},
        {"name": "컴퓨터네트워크", "hours": 2, "professor": "정하윤", "type": "일반", "grade": 3, "students": 40},
        {"name": "머신러닝", "hours": 3, "professor": "박지훈", "type": "컴퓨터", "grade": 4, "students": 20},
        {"name": "알고리즘1", "hours": 3, "professor": "김민준", "type": "컴퓨터", "grade": 3, "students": 25},
        {"name": "데이터베이스1", "hours": 2, "professor": "이서연", "type": "일반", "grade": 2, "students": 35},
        {"name": "운영체제1", "hours": 3, "professor": "최유진", "type": "컴퓨터", "grade": 3, "students": 28},
        {"name": "컴퓨터네트워크1", "hours": 2, "professor": "정하윤", "type": "일반", "grade": 3, "students": 40},
        {"name": "머신러닝1", "hours": 3, "professor": "박지훈", "type": "컴퓨터", "grade": 4, "students": 20},
        {"name": "알고리즘2", "hours": 6, "professor": "조상1", "type": "컴퓨터", "grade": 3, "students": 25},
        {"name": "알고리즘3", "hours": 6, "professor": "조상2", "type": "컴퓨터", "grade": 3, "students": 25},
        {"name": "알고리즘4", "hours": 6, "professor": "조강2", "type": "컴퓨터", "grade": 3, "students": 25},
    ]
    return courses

# ✅ classrooms를 딕셔너리로 정의
classrooms = {
    "1215": {"type": "컴퓨터", "capacity": 30},
    "1216": {"type": "일반", "capacity": 40},
    "1217": {"type": "컴퓨터", "capacity": 30},
    "1416": {"type": "일반", "capacity": 50},
    "1218": {"type": "컴퓨터", "capacity": 30}
}

# 기본 설정
courses = create_courses()
time_slots = [f"{h:02d}:00" for h in range(9, 18)]  # 09:00-17:00


In [3]:
time_slots

['09:00',
 '10:00',
 '11:00',
 '12:00',
 '13:00',
 '14:00',
 '15:00',
 '16:00',
 '17:00']

In [4]:

def solve_timetable_ip():
    """정수계획법을 사용한 강의 시간표 배정"""

    print("🚀 정수계획법 기반 강의 시간표 배정 시작...")
    print("=" * 60)

    # 문제 정의
    prob = pulp.LpProblem("강의_시간표_배정", pulp.LpMinimize)

    # 인덱스 생성
    course_indices = {course['name']: i for i, course in enumerate(courses)}
    time_indices = {time: i for i, time in enumerate(time_slots)}
    room_indices = {room: i for i, room in enumerate(classrooms.keys())}

    # 결정 변수: x[i,t,r] = 강의 i가 시간 t에 강의실 r에 배정되면 1, 아니면 0
    x = {}
    for i, course in enumerate(courses):
        for t, time_slot in enumerate(time_slots):
            for r, classroom in enumerate(classrooms.keys()):
                var_name = f"x_{i}_{t}_{r}"
                x[(i, t, r)] = pulp.LpVariable(var_name, cat='Binary')

    # 목적함수: 단순히 제약 조건 만족을 목표로
    prob += 0

    print(f"📊 문제 규모:")
    print(f"  • 강의 수: {len(courses)}개")
    print(f"  • 시간 슬롯: {len(time_slots)}개")
    print(f"  • 강의실 수: {len(classrooms)}개")
    print(f"  • 결정 변수 수: {len(courses) * len(time_slots) * len(classrooms)}개")

    # 제약 조건 1: 각 강의는 정해진 시수만큼 배정
    for i, course in enumerate(courses):
        prob += (
            pulp.lpSum([x[(i, t, r)] for t in range(len(time_slots)) for r in range(len(classrooms))])
            == course['hours'],
            f"시수_제약_{course['name']}"
        )

    # 제약 조건 2: 한 시간에 한 강의실에는 최대 하나의 강의만
    for t in range(len(time_slots)):
        for r, classroom in enumerate(classrooms.keys()):
            prob += (
                pulp.lpSum([x[(i, t, r)] for i in range(len(courses))]) <= 1,
                f"강의실_제약_{time_slots[t]}_{classroom}"
            )

    # 제약 조건 3: 교수는 동시에 한 강의만 가능
    prof_courses = {}
    for i, course in enumerate(courses):
        prof = course['professor']
        if prof not in prof_courses:
            prof_courses[prof] = []
        prof_courses[prof].append(i)

    print(f"\n👨‍🏫 교수별 담당 과목:")
    for professor, course_ids in prof_courses.items():
        course_names = [courses[i]['name'] for i in course_ids]
        total_hours = sum(courses[i]['hours'] for i in course_ids)
        print(f"  • {professor}: {len(course_names)}과목, {total_hours}시간")
        print(f"    → {', '.join(course_names)}")

    for professor, course_ids in prof_courses.items():
        if len(course_ids) > 1:
            for t in range(len(time_slots)):
                prob += (
                    pulp.lpSum([x[(i, t, r)] for i in course_ids for r in range(len(classrooms))]) <= 1,
                    f"교수_제약_{professor}_{time_slots[t]}"
                )

    # 제약 조건 4: 각 강의는 한 시간에 하나의 강의실만
    for i in range(len(courses)):
        for t in range(len(time_slots)):
            prob += (
                pulp.lpSum([x[(i, t, r)] for r in range(len(classrooms))]) <= 1,
                f"강의_시간_제약_{courses[i]['name']}_{time_slots[t]}"
            )

    print(f"\n⚙️  제약 조건 설정 완료")

    # 문제 해결
    solver = pulp.PULP_CBC_CMD(timeLimit=30, msg=True)
    prob.solve(solver)

    status = pulp.LpStatus[prob.status]
    print(f"📊 해결 상태: {status}")

    if prob.status == pulp.LpStatusOptimal or prob.status == pulp.LpStatusFeasible:
        return prob, x, status
    else:
        return prob, None, status

def display_ip_results(prob, x, status):
    """정수계획법 결과 출력"""

    print("\n" + "="*60)
    print("🎯 정수계획법 기반 강의 시간표 배정 결과")
    print("="*60)

    if x is None or status not in ['Optimal', 'Feasible']:
        print(f"\n❌ 해결책을 찾을 수 없습니다. 상태: {status}")
        return

    # 해결책 추출
    solution = {}
    for i, course in enumerate(courses):
        for t, time_slot in enumerate(time_slots):
            for r, classroom in enumerate(classrooms.keys()):
                if x[(i, t, r)].varValue == 1:
                    if course['name'] not in solution:
                        solution[course['name']] = []
                    solution[course['name']].append((time_slot, classroom))

    # 학년별 결과 출력
    print(f"\n📚 학년별 강의 배정 현황:")
    print("-" * 50)

    courses_by_grade = {}
    for course in courses:
        grade = course.get('grade', '미정')
        if grade not in courses_by_grade:
            courses_by_grade[grade] = []
        courses_by_grade[grade].append(course)

    total_assigned = 0
    total_required = 0

    for grade in sorted(courses_by_grade.keys()):
        print(f"\n🎓 {grade}학년:")
        print("=" * 30)

        for course in courses_by_grade[grade]:
            type_icon = "💻" if course['type'] == "컴퓨터" else "📖"
            assigned_hours = len(solution.get(course['name'], []))
            required_hours = course['hours']

            total_assigned += assigned_hours
            total_required += required_hours

            print(f"\n  {type_icon} {course['name']} ({course['professor']})")
            print(f"      필요: {required_hours}시간, 배정: {assigned_hours}시간, 수강생: {course['students']}명")

            if course['name'] in solution:
                for time_slot, classroom in sorted(solution[course['name']]):
                    room_info = classrooms[classroom]
                    room_icon = "💻" if room_info['type'] == "컴퓨터" else "📖"
                    print(f"      📍 {time_slot} | {room_icon} {classroom} ({room_info['capacity']}명)")
            else:
                print(f"      ⚠️ 배정되지 않음")

    # 강의실별 시간표
    print(f"\n\n🏛️ 강의실별 시간표:")
    print("=" * 60)

    for classroom, room_info in classrooms.items():
        room_icon = "💻" if room_info['type'] == "컴퓨터" else "📖"
        print(f"\n{room_icon} 강의실 {classroom} ({room_info['type']}, {room_info['capacity']}명)")
        print("-" * 40)

        schedule = {}
        for course_name, assignments in solution.items():
            for time_slot, room in assignments:
                if room == classroom:
                    course_obj = next(c for c in courses if c['name'] == course_name)
                    prof = course_obj['professor']
                    students = course_obj['students']
                    schedule[time_slot] = f"{course_name} ({prof}, {students}명)"

        if schedule:
            for time_slot in time_slots:
                if time_slot in schedule:
                    print(f"  {time_slot}: {schedule[time_slot]}")
                else:
                    print(f"  {time_slot}: 🔓 비어있음")
        else:
            print("  📭 배정된 강의 없음")

    # 강의실 활용률 분석
    print(f"\n\n🏛️ 강의실 활용률 분석:")
    print("=" * 45)

    for classroom, room_info in classrooms.items():
        used_slots = sum(1 for assignments in solution.values()
                        for _, room in assignments if room == classroom)
        utilization = (used_slots / len(time_slots)) * 100
        room_icon = "💻" if room_info['type'] == "컴퓨터" else "📖"

        print(f"  {room_icon} {classroom}: {used_slots}/{len(time_slots)}시간 ({utilization:.1f}%)")

    # 최종 통계
    success_rate = (total_assigned / total_required * 100) if total_required > 0 else 0

    print(f"\n\n📊 배정 통계:")
    print("=" * 40)
    print(f"  • 총 과목 수: {len(courses)}개")
    print(f"  • 필요 총 시간: {total_required}시간")
    print(f"  • 배정된 시간: {total_assigned}시간")
    print(f"  • 배정 성공률: {success_rate:.1f}%")
    print(f"  • 사용된 강의실: {len([r for r in classrooms if any(room == r for assignments in solution.values() for _, room in assignments)])}개/{len(classrooms)}개")
    print(f"  • 해결 상태: {status}")

    comp_courses = [c for c in courses if c['type'] == '컴퓨터']
    general_courses = [c for c in courses if c['type'] == '일반']
    comp_rooms_count = len([r for r in classrooms.values() if r['type'] == '컴퓨터'])
    general_rooms_count = len([r for r in classrooms.values() if r['type'] == '일반'])

    print(f"\n  📊 과목-강의실 매칭:")
    print(f"  • 💻 컴퓨터 과목: {len(comp_courses)}개 → 컴퓨터실: {comp_rooms_count}개")
    print(f"  • 📖 일반 과목: {len(general_courses)}개 → 일반강의실: {general_rooms_count}개")

    if prob.objective is not None:
        print(f"  • 목적함수 값: {pulp.value(prob.objective)}")

def print_course_summary():
    """교과목 현황 요약"""
    print("📚 교과목 현황")
    print("=" * 40)

    grade_stats = {}
    type_stats = {"컴퓨터": 0, "일반": 0}
    total_hours = 0
    professors = set()

    for course in courses:
        grade = course.get('grade', '미정')
        if grade not in grade_stats:
            grade_stats[grade] = {'count': 0, 'hours': 0}

        grade_stats[grade]['count'] += 1
        grade_stats[grade]['hours'] += course['hours']
        type_stats[course['type']] += 1
        total_hours += course['hours']
        professors.add(course['professor'])

    for grade in sorted(grade_stats.keys()):
        stats = grade_stats[grade]
        print(f"  🎓 {grade}학년: {stats['count']}과목, {stats['hours']}시간")

    print(f"\n  📊 전체: {len(courses)}과목, {total_hours}시간")
    print(f"  💻 컴퓨터과목: {type_stats['컴퓨터']}개")
    print(f"  📖 일반과목: {type_stats['일반']}개")
    print(f"  👨‍🏫 참여교수: {len(professors)}명")

    print(f"\n  🏛️ 강의실: {len(classrooms)}개")
    print(f"  ⏰ 시간대: {len(time_slots)}시간 ({time_slots[0]}-{time_slots[-1]})")
    print(f"  📦 가용시간: {len(classrooms) * len(time_slots)}시간")

def main():
    """메인 실행 함수"""
    print("🎯 정수계획법 기반 강의 시간표 자동 배정 시스템\n")

    # 교과목 현황 출력
    print_course_summary()

    try:
        # 정수계획법으로 시간표 배정
        prob, x, status = solve_timetable_ip()

        # 결과 출력
        display_ip_results(prob, x, status)

    except Exception as e:
        print(f"\n❌ 오류 발생: {str(e)}")
        print("💡 PuLP 라이브러리가 설치되어 있는지 확인해주세요:")
        print("   pip install pulp")

if __name__ == "__main__":
    main()


🎯 정수계획법 기반 강의 시간표 자동 배정 시스템

📚 교과목 현황
  🎓 2학년: 2과목, 4시간
  🎓 3학년: 9과목, 34시간
  🎓 4학년: 2과목, 6시간

  📊 전체: 13과목, 44시간
  💻 컴퓨터과목: 9개
  📖 일반과목: 4개
  👨‍🏫 참여교수: 8명

  🏛️ 강의실: 5개
  ⏰ 시간대: 9시간 (09:00-17:00)
  📦 가용시간: 45시간
🚀 정수계획법 기반 강의 시간표 배정 시작...
📊 문제 규모:
  • 강의 수: 13개
  • 시간 슬롯: 9개
  • 강의실 수: 5개
  • 결정 변수 수: 585개

👨‍🏫 교수별 담당 과목:
  • 김민준: 2과목, 6시간
    → 알고리즘, 알고리즘1
  • 이서연: 2과목, 4시간
    → 데이터베이스, 데이터베이스1
  • 최유진: 2과목, 6시간
    → 운영체제, 운영체제1
  • 정하윤: 2과목, 4시간
    → 컴퓨터네트워크, 컴퓨터네트워크1
  • 박지훈: 2과목, 6시간
    → 머신러닝, 머신러닝1
  • 조상1: 1과목, 6시간
    → 알고리즘2
  • 조상2: 1과목, 6시간
    → 알고리즘3
  • 조강2: 1과목, 6시간
    → 알고리즘4

⚙️  제약 조건 설정 완료
📊 해결 상태: Optimal

🎯 정수계획법 기반 강의 시간표 배정 결과

📚 학년별 강의 배정 현황:
--------------------------------------------------

🎓 2학년:

  📖 데이터베이스 (이서연)
      필요: 2시간, 배정: 2시간, 수강생: 35명
      📍 11:00 | 💻 1218 (30명)
      📍 15:00 | 💻 1215 (30명)

  📖 데이터베이스1 (이서연)
      필요: 2시간, 배정: 2시간, 수강생: 35명
      📍 09:00 | 💻 1218 (30명)
      📍 10:00 | 💻 1217 (30명)

🎓 3학년:

  💻 알고리즘 (김민준)
      필요: 3시간, 배정: 

### 과목은 시수기준으로 연속하여 진행되어야 함